In [ ]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from rioxarray import merge
from rasterio.enums import Resampling
from osgeo import gdal
from joblib import Parallel, delayed
import sqlite3
%matplotlib widget

In [2]:
""" Note: some regions need to be merged together, e.g. 1-2, 13-14-15.
I can also merge all of them together"""

' Note: some regions need to be merged together, e.g. 1-2, 13-14-15.\nI can also merge all of them together'

In [6]:
def topo_colormap():
    colors = [
        (0.00, "#0c00c1"),   # deep ocean 
        (0.17, "#001CD8"),
        (0.33, "#1FB3FF"),   # mid ocean
        (0.50, "#BEE5FB"),   # shallow ocean
        (0.50, "#0F7127"),   # land start #1c8c37
        (0.67, "#DBFBBC"),   # bianchino smielenso #e2fcc7
        (0.83, "#8F7C21"),    # #9f892d
        (1.00, "#4B1A01")
    ]
    return LinearSegmentedColormap.from_list("blue_brown", colors, N=256)

In [7]:
class info:
    save_variable = 'bed' # ('ice', 'bed')
    rgi = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    version = '70G' # ('62', '70G')
    ICE_CMAP = plt.get_cmap('turbo', 256)
    ICE_MIN_VAL_MAPBOX = 0
    ICE_MAX_VAL_MAPBOX = 800

    BED_CMAP = topo_colormap()
    BED_MIN_VAL_MAPBOX = -500
    BED_MAX_VAL_MAPBOX = 1500

    #BASE_PATH = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250612/RGI{version}"
    BASE_PATH = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20251009/RGI{version}"
    #BASE_PATH = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250411/RGI{version}"
    #BASE_PATH = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250425/RGI{version}"
    #BASE_PATH = f"/media/maffe/nvme/iceboost_global_deploy/iceboost_20250606/RGI{version}"
    n_jobs = 20 # 20

In [8]:
# Generate paths for each RGI
PATH_TIFFS_IN = [f"{info.BASE_PATH}/rgi{r}" for r in info.rgi]

# Collect all TIFF filenames from all directories
tif_filenames = []
for path in PATH_TIFFS_IN:
    tif_filenames.extend(glob.glob(f'{path}/*.tif'))  # Stores full paths
print(f"We will process {len(tif_filenames)} files from rgi {info.rgi}")

We will process 274531 files from rgi [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [9]:
#tif_filenames = tif_filenames[0:3]

In [10]:
print(tif_filenames[0])
print(len(tif_filenames))

/media/maffe/nvme/iceboost_global_deploy/iceboost_20251009/RGI70G/rgi1/RGI2000-v7.0-G-01-01477.tif
274531


In [11]:
# Process individual TIF files and save them as .tif in the temporary /tmp folder (we will remove them in the end)

#def process_tif(input_tif_name, cmap, min_val_mapbox, max_val_mapbox):
def process_tif(input_tif_name, info):
    """Function to process a single tif file."""
    
    ice_cmap=info.ICE_CMAP
    ice_min_val_mapbox=info.ICE_MIN_VAL_MAPBOX
    ice_max_val_mapbox=info.ICE_MAX_VAL_MAPBOX

    bed_cmap=info.BED_CMAP
    bed_min_val_mapbox=info.BED_MIN_VAL_MAPBOX
    bed_max_val_mapbox=info.BED_MAX_VAL_MAPBOX
    
    # band=1 is ice thickness. h_wgs84 is band=3
    tif = rioxarray.open_rasterio(f"{input_tif_name}")  # projection can be EPSG:4326 or 3031 or 3413
    #tif_3857 = tif.rio.reproject(dst_crs="EPSG:3857", resampling=Resampling.bilinear)
    #print(tif.attrs)
    
    ice = tif.sel(band=1)
    h_wgs84 = tif.sel(band=4) #band=3
    n_geoid = tif.sel(band=5) #band=4
    h_ortho = h_wgs84 - n_geoid
    
    ice = ice.squeeze().rio.reproject(dst_crs="EPSG:3857", resampling=Resampling.bilinear)
    #h_wgs84 = h_wgs84.squeeze().rio.reproject(dst_crs="EPSG:3857", resampling=Resampling.bilinear)
    h_ortho = h_ortho.squeeze().rio.reproject(dst_crs="EPSG:3857", resampling=Resampling.bilinear)
    # TODO: instead of h_wgs84 I should use EIGEN-6C4
    #bed = h_wgs84 - ice
    bed = h_ortho - ice
    ice = ice.clip(ice_min_val_mapbox, ice_max_val_mapbox).fillna(0)

    ice_bad_mask = (ice <= 0) | ice.isnull()
    bed = bed.clip(bed_min_val_mapbox, bed_max_val_mapbox)
    bed = bed.where(~ice_bad_mask)
    
    #fig, (ax1, ax2, ax3) = plt.subplots(1,3)
    #ice.plot(ax=ax1)
    #h_wgs84.plot(ax=ax2)
    #bed.plot(ax=ax3)
    #plt.show()

    # normalize ice to [0, 255]
    ice_rgba_data = ice_cmap(ice.values / ice_max_val_mapbox) * 255
    # normalize bed to [0, 255]
    norm_for_bed = TwoSlopeNorm(vmin=bed_min_val_mapbox, vcenter=0, vmax=bed_max_val_mapbox)
    bed_rgba_data = bed_cmap(norm_for_bed(bed.values)) * 255
    #bed_rgba_data = bed_cmap((bed.values - bed_min_val_mapbox) / (bed_max_val_mapbox - bed_min_val_mapbox)) * 255
    
    # Set alpha to fully transparent (0) where data is bad (0)
    ice_rgba_data[ice.values == 0] = [0, 0, 0, 0]  # Fully transparent for bad data
    bed_rgba_data[np.isnan(bed.values)] = [0, 0, 0, 0]
    
    ice_rgba_data = ice_rgba_data.astype(np.uint8)
    bed_rgba_data = bed_rgba_data.astype(np.uint8)
    
    ice_rgb_data_array = xr.DataArray(
        ice_rgba_data, dims=('y', 'x', 'band'),
        coords={'x': ice.coords['x'], 'y': ice.coords['y']}
    ).transpose('band', 'y', 'x')

    bed_rgb_data_array = xr.DataArray(
        bed_rgba_data, dims=('y', 'x', 'band'),
        coords={'x': bed.coords['x'], 'y': bed.coords['y']}
    ).transpose('band', 'y', 'x')
    
    ice_rgb_data_array.rio.write_crs("EPSG:3857", inplace=True).rio.write_nodata(0, inplace=True)
    bed_rgb_data_array.rio.write_crs("EPSG:3857", inplace=True).rio.write_nodata(None, inplace=True)
    
    # Skip 1-pixel glaciers
    if ice_rgb_data_array.shape[1] == 1 or ice_rgb_data_array.shape[2] == 1:
        return None
    
    # Save to a temporary file and return its path
    temp_file = tempfile.NamedTemporaryFile(suffix=".tif", delete=False).name
    
    if info.save_variable == 'ice':
        ice_rgb_data_array.rio.to_raster(temp_file, compress="deflate")
    elif info.save_variable == 'bed':
        bed_rgb_data_array.rio.to_raster(temp_file, compress="deflate")
    else:
        raise ValueError('Bad variable indicated in info. Exit.')

    return temp_file

# Parallel processing
n_jobs = info.n_jobs
processed_tifs = Parallel(n_jobs=n_jobs)(
    delayed(process_tif)(input_tif_name=tif_name,
                         info=info
                         )
    for tif_name in tqdm(tif_filenames, total=len(tif_filenames))
)

# Remove None values (from skipped 1-pixel glaciers)
processed_tifs = [tif for tif in processed_tifs if tif is not None]
print(f"We have no. tif to process: {len(processed_tifs)}")

 17%|█████▉                             | 46840/274531 [03:12<25:37, 148.14it/s]/home/maffe/anaconda3/envs/skynetenv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████████████████████████████| 274531/274531 [29:28<00:00, 155.20it/s]


We have no. tif to process: 274531


In [12]:
date_n_time = time.strftime("%Y%m%d", time.localtime())
vrt_path = f"{info.save_variable}_v{info.version}_rgi_{'_'.join(map(str, info.rgi))}_{date_n_time}.vrt" # Temporary VRT file # e.g. rgi_6.vrt
mbtiles_output = f"{info.save_variable}_v{info.version}_rgi_{'_'.join(map(str, info.rgi))}_{date_n_time}.mbtiles" # Output MBTiles file
print(vrt_path, mbtiles_output)

# Step 1: Create a Virtual Raster (VRT)
try:
    print(f'Begin VRT creation')
    t1_0 = time.time()
    gdal.BuildVRT(vrt_path, processed_tifs)
    if os.path.exists(vrt_path):
        print(f"{vrt_path} successfully created.")
        vrt_size = os.path.getsize(vrt_path)
        print(f"Size of {vrt_path}: {vrt_size / 1024:.2f} KB")
    else:
        raise FileNotFoundError(f"Failed to create {vrt_path}.")
    print(f"End VRT creation in {time.time()-t1_0:.2f} sec")
    

finally:
    print('FINE')

bed_v70G_rgi_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20251111.vrt bed_v70G_rgi_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20251111.mbtiles
Begin VRT creation


/home/maffe/anaconda3/envs/skynetenv/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


bed_v70G_rgi_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20251111.vrt successfully created.
Size of bed_v70G_rgi_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20251111.vrt: 506403.73 KB
End VRT creation in 248.54 sec
FINE


In [13]:
# Step 2: Convert VRT to MBTiles using gdal_translate
translate_command = [
    'gdal_translate',
    '-mask', '4',
    '-of', 'MBTILES',  # Output format MBTiles
    '-co', 'TILE_FORMAT=PNG',  # Tile format (PNG or JPEG)
    '-co', 'ZOOM_LEVEL_STRATEGY=LOWER',  # The nearest lower zoom level (less detailed) is chosen.
    vrt_path, mbtiles_output
]

print(f'Begin MBTiles generation')
t2_0 = time.time()
subprocess.run(translate_command, check=True)

# Step 2.1: Retrieve Maximum Zoom Level from MBTiles
conn = sqlite3.connect(mbtiles_output)
cursor = conn.cursor()
cursor.execute("SELECT value FROM metadata WHERE name='maxzoom';")
max_zoom = cursor.fetchone()
conn.close()

if max_zoom:
    max_zoom = int(max_zoom[0])
    print(f"We have fetched the maxzoom as being: {max_zoom}")
else:
    raise ValueError("Could not determine maxzoom from MBTiles metadata.")


# Generate all zoom levels down to 0
overview_levels = [2**i for i in range(max_zoom, 0, -1)]  # E.g., [2^11, 2^10, ..., 2]
overview_levels_str = ' '.join(map(str, overview_levels))

print(f"Generating overviews for zoom levels: {overview_levels_str}")
subprocess.run(['gdaladdo', '-r', 'bilinear', mbtiles_output] + overview_levels_str.split(), check=True)
#subprocess.run(['gdaladdo', '-r', 'bilinear', mbtiles_output, '2', '4', '8', '16', '32', '64', '128'], check=True)
print(f"MBTiles file created at: {mbtiles_output} in {time.time()-t2_0:.2f} sec")

# Step 3: Update minzoom in Metadata
print("Updating minzoom in MBTiles metadata...")
min_zoom = 0  # Set minimum zoom to 0

conn = sqlite3.connect(mbtiles_output)
cursor = conn.cursor()
cursor.execute("UPDATE metadata SET value = ? WHERE name = 'minzoom';", (min_zoom,))
conn.commit()
conn.close()

print(f"Updated minzoom to {min_zoom}.")

Begin MBTiles generation
Input file size is 561741, 463140
0...10...20...30...40...50...60...70...80...90...100 - done.
We have fetched the maxzoom as being: 11
Generating overviews for zoom levels: 2048 1024 512 256 128 64 32 16 8 4 2
0...10...20...30...40...50...60...70...80...90...100 - done.
MBTiles file created at: bed_v70G_rgi_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16_17_18_19_20251111.mbtiles in 69852.64 sec
Updating minzoom in MBTiles metadata...
Updated minzoom to 0.


In [15]:
# If something is left in /tmp remove any .tif file (it should not be necessary)

# Find all .tif files in /tmp
junk_tif_files = glob.glob("/tmp/*.tif")

# Calculate total size of the .tif files
total_size_mb = sum(os.path.getsize(f) for f in junk_tif_files if os.path.isfile(f)) / (1024 ** 2)
num_files = len(junk_tif_files)

# Delete each file
for file_path in junk_tif_files:
    os.remove(file_path)
    #print(f"Deleted {file_path}")

print(f"Deleted {num_files} .tif files, freeing {total_size_mb:.2f} MB.")

Deleted 0 .tif files, freeing 0.00 MB.


In [2]:
# Generate BedMachine v3.7


MAX_VAL_MAPBOX = 800
cmap = plt.get_cmap('turbo', 256)

bedmachine_file = '/media/maffe/nvme/Antarctica_NSIDC/thickness/NSIDC-0756/BedMachineAntarctica-v3.nc'
bedmachine = rioxarray.open_rasterio(bedmachine_file, masked=False)
bedmachine = bedmachine.thickness
bedmachine.values = np.where((bedmachine.values == bedmachine.rio.nodata), 0.0, bedmachine.values)
bedmachine = bedmachine.squeeze()
bedmachine = bedmachine.rio.reproject(dst_crs="EPSG:3857", resampling=Resampling.bilinear).clip(0, MAX_VAL_MAPBOX).fillna(0)

rgba_data = cmap(bedmachine.values / MAX_VAL_MAPBOX) * 255

rgba_data[bedmachine.values == 0] = [0, 0, 0, 0]  # Fully transparent for bad data

rgba_data = rgba_data.astype(np.uint8)

rgb_data_array = xr.DataArray(
    rgba_data, dims=('y', 'x', 'band'),
    coords={'x': bedmachine.coords['x'], 'y': bedmachine.coords['y']}
).transpose('band', 'y', 'x')

rgb_data_array.rio.write_crs("EPSG:3857", inplace=True)
rgb_data_array.rio.write_nodata(0, inplace=True)

<xarray.DataArray (band: 4, y: 6161, x: 17821)>
array([[[122, 122, 122, ..., 122, 122, 122],
        [122, 122, 122, ..., 122, 122, 122],
        [122, 122, 122, ..., 122, 122, 122],
        ...,
        [ 50,  50,  50, ...,  52,  52,  52],
        [ 50,  50,  50, ...,  52,  52,  52],
        [ 52,  50,  50, ...,  52,  52,  52]],

       [[  4,   4,   4, ...,   4,   4,   4],
        [  4,   4,   4, ...,   4,   4,   4],
        [  4,   4,   4, ...,   4,   4,   4],
        ...,
        [ 24,  24,  24, ...,  27,  27,  27],
        [ 24,  24,  24, ...,  27,  27,  27],
        [ 27,  24,  24, ...,  27,  27,  27]],

       [[  2,   2,   2, ...,   2,   2,   2],
        [  2,   2,   2, ...,   2,   2,   2],
        [  2,   2,   2, ...,   2,   2,   2],
        ...,
        [ 74,  74,  74, ...,  81,  81,  81],
        [ 74,  74,  74, ...,  81,  81,  81],
        [ 81,  74,  74, ...,  81,  81,  81]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8)
Coordinates:
  * x            (x) float64 -2.004e+07 -2.003e+07 ... 2.003e+07 2.004e+07
  * y            (y) float64 -6.185e+06 -6.187e+06 ... -2.004e+07 -2.004e+07
    spatial_ref  int64 0
Dimensions without coordinates: band
Attributes:
    _FillValue:  0

In [3]:
# Save to a temporary file and return its path
temp_file = tempfile.NamedTemporaryFile(suffix=".tif", delete=False).name
rgb_data_array.rio.to_raster(temp_file, compress="deflate")

print(temp_file)

processed_tifs = [temp_file]

/tmp/tmpim_bossg.tif


In [4]:
vrt_path = "temp.vrt"  # Temporary VRT file
mbtiles_output = f"bedmachine_v37.mbtiles"  # Output MBTiles file

try:
    # Step 1: Create a Virtual Raster (VRT)
    print(f'Begin VRT creation')
    t1_0 = time.time()
    gdal.BuildVRT(vrt_path, processed_tifs)
    if os.path.exists(vrt_path):
        print(f"{vrt_path} successfully created.")
        vrt_size = os.path.getsize(vrt_path)
        print(f"Size of {vrt_path}: {vrt_size / 1024:.2f} KB")
    else:
        raise FileNotFoundError(f"Failed to create {vrt_path}.")
    print(f"End VRT creation in {time.time()-t1_0:.2f} sec")
    
    # Step 2: Convert VRT to MBTiles using gdal_translate
    translate_command = [
        'gdal_translate',
        '-mask', '4',
        '-of', 'MBTILES',  # Output format MBTiles
        '-co', 'TILE_FORMAT=PNG',  # Tile format (PNG or JPEG)
        '-co', 'ZOOM_LEVEL_STRATEGY=LOWER',  # The nearest lower zoom level (less detailed) is chosen.
        vrt_path, mbtiles_output
    ]

    print(f'Begin MBTiles generation')
    t2_0 = time.time()
    subprocess.run(translate_command, check=True)
    subprocess.run(['gdaladdo', '-r', 'bilinear', mbtiles_output, '2', '4', '8', '16', '32', '64', '128'], check=True)
    print(f"MBTiles file created at: {mbtiles_output} in {time.time()-t2_0:.2f} sec")

    # Step 3: Update minzoom and maxzoom in Metadata
    print("Updating minzoom and maxzoom in MBTiles metadata...")
    min_zoom = 3  # Set your desired minimum zoom level
    max_zoom = 6  # Set your desired maximum zoom level
    conn = sqlite3.connect(mbtiles_output)
    cursor = conn.cursor()
    cursor.execute("UPDATE metadata SET value = ? WHERE name = 'minzoom';", (min_zoom,))
    cursor.execute("UPDATE metadata SET value = ? WHERE name = 'maxzoom';", (max_zoom,))
    conn.commit()
    conn.close()
    print(f"Updated minzoom to {min_zoom} and maxzoom to {max_zoom} in metadata.")

finally:
    # Cleanup temporary files
    for file_path in [vrt_path]:
        if os.path.exists(file_path):
            os.remove(file_path)
            print(f"Deleted {file_path}.")

Begin VRT creation
temp.vrt successfully created.
Size of temp.vrt: 3.10 KB
End VRT creation in 0.00 sec
Begin MBTiles generation
Input file size is 17821, 6161
0

/home/maffe/anaconda3/envs/skynetenv/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
MBTiles file created at: bedmachine_v37.mbtiles in 25.18 sec
Updating minzoom and maxzoom in MBTiles metadata...
Updated minzoom to 3 and maxzoom to 6 in metadata.
Deleted temp.vrt.
